# 3. Interactive plotting of 4D data

Subsurface Hackathon 2017, Paris.

*Joseph Barraud and Martin Bentley*

---
This notebook describes the work that was conducted by our team during the [Subsurface Hackathon in Paris](https://agilescientific.com/paris/) (10-11 June 2017). 

This is the third step of a short study on groundwater levels in the Netherlands. Please see the [first notebook](.\1. Levels data loading.ipynb) for an introduction about the data.

In [1]:
import os
import numpy as np
from scipy import interpolate
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import datetime
% matplotlib inline

# import ipywidgets
from ipywidgets import *

### Options

The images generated in this notebook have a transparent background. Unfortunately, when you try and copy those images and paste them into a program like PowerPoint, you will get a black background instead.

The following change fixes this problem.

In [2]:
mpl.rcParams['figure.facecolor'] = 'white'

In [3]:
# pandas options
pd.options.display.max_rows = 6

## Data loading

The data were previously imported from csv files into a pandas dataframe (see [first notebook](.\1. Levels data loading.ipynb)).

In [4]:
all_depths = pd.read_pickle(r'..\data\Groundwater-Levels\all_depths_v1.pkl.gz')

### Data preparation

Let's redo quickly what was described in the [previous notebook](.\2. Create surfaces from data points), i.e. resample the data in time in order to get aggregated data for each month.

In [5]:
all_depths1 = all_depths.loc[(slice(None),1),:].reset_index(1, drop=True)
all_depths1_permonth = all_depths1.reset_index(0).groupby('well').resample('MS').mean()

## Interactive plotting

We have 4D data: three spatial dimensions and time. So one way to explore this dataset is to get a map of the water depth for any date in our database. We can make this plot interactive by having a cursor that would control time. This is quite easy to do with a library called [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/).

All we need is a function that will create the surface and make the plot. For that, we can simply copy we have done in the previous notebook. Then we use [interact](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html) to link the function with a couple of widgets.

In [6]:
# first define the dimensions of the grid for the depth surface
x_min = all_depths.x.min()
x_max = all_depths.x.max()
y_min = all_depths.y.min()
y_max = all_depths.y.max()

cellsize = 100
vector_x = np.arange(x_min-2*cellsize, x_max+3*cellsize, cellsize)
vector_y = np.arange(y_min-2*cellsize, y_max+3*cellsize, cellsize)
grid_x, grid_y = np.meshgrid(vector_x,vector_y)
extent1 = (vector_x.min(),vector_x.max(), vector_y.min(), vector_y.max())

In [7]:
def plot_surface(date='1996-01-01', method='nearest', add_wells=True):
    # selection and gridding
    dfa = all_depths1_permonth.loc[(slice(None),date),:].dropna()
    points = dfa[['x','y']]
    values = dfa.depth
    grid1 = interpolate.griddata(points, values, (grid_x, np.flipud(grid_y)), method=method)
    # plotting
    fig,ax = plt.subplots(figsize=(16,12))
    plt.imshow(grid1,extent=extent1,origin='upper', cmap='coolwarm_r',
               vmin=all_depths1.depth.quantile(0.04) ,
               vmax=all_depths1.depth.quantile(0.96))  # fix the depth range to fix the colormap
    if add_wells: 
        dfa.plot.scatter(x='x', y='y', c='none', s=50,edgecolors='k', ax=ax, colorbar=False)
    # 
    plt.colorbar(shrink=0.5)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_title(date)
    plt.show() # this is necessary to ensure the same plot is updated as we change the date with the cursor

In [8]:
# create a list of dates (every first day of the month from 1955 to 2015)
dates = [datetime.date(y,m,1) for y in range(1955,2016) for m in range(1,13) ]
# create a widget with a slider for the data
dateWidget = SelectionSlider(
                options=dates,
                description='Date')
# make the slider wider
dateWidget.layout.width = '500px'
# make a couple of radio buttons for the method selection
methodWidget = RadioButtons(
            options=['nearest', 'linear'], value='nearest',
            description='Interpolation:')
# run widget with the interact function (note how the function automatically creates a tick box for the add_wells parameter)
interact(plot_surface, date=dateWidget, method=methodWidget)

<function __main__.plot_surface>

Finally, this last example combines a dropdown menu with a plot of water depth vs time. So we have a simple way to examine the data for each well in our database.

In [9]:
def plot_depth_vs_time(well='B24F0005'):
    # selection 
    dfa = all_depths1_permonth.loc[well]
    
    # plot
    fig,ax = plt.subplots(figsize=(12,6))
    ax.plot(dfa.depth)
    ax.set_xlabel('')
    ax.set_ylabel('mean depth (cm w.r.t. NAP)')
    ax.set_xlim(pd.to_datetime('1950'), pd.to_datetime('2017'))
    ax.set_ylim(-750,750)
    ax.xaxis.set_major_locator(mdates.YearLocator(10))
    ax.grid()
    plt.show()

In [10]:
# create a list of all the wells
well_list = all_depths1.index.get_level_values(0).unique().tolist()
# create a widget with a dropdown menu
wellWidget = Dropdown(options=well_list, value=well_list[0],
                description='Well ID')

# run widget with the interact function
interact(plot_depth_vs_time, well=wellWidget)

<function __main__.plot_depth_vs_time>